In [1]:
%matplotlib inline
import numpy as np
import glob
from os.path import join, basename
from astropy.io import fits
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.ticker as ticker

## Modify CrosstalkColumnTask

Develop a way to programmatically find aggressor amplifier and column numbers for crosstalk analysis, rather than relying on user specification.

In [21]:
from sqlalchemy.orm.exc import NoResultFound

import lsst.eotest.image_utils as imutils
from lsst.eotest.sensor.MaskedCCD import MaskedCCD
from lsst.eotest.sensor.BrightPixels import BrightPixels
from lsst.eotest.sensor.crosstalk import column_mean

from mixcoatl.crosstalk import CrosstalkMatrix, make_stamp, crosstalk_fit
from mixcoatl.utils import AMP2SEG
from mixcoatl.database import Sensor, Segment, Result, db_session

## Debug stuff
import lsst.geom as lsstGeom
import lsst.afw.math as afwMath

In [17]:
main_dir = '/project/bootcamp/cslage/e2v_fits_files/satellite/20200902-injectdata'
#main_dir = '/project/bootcamp/cslage/e2v_fits_files/satellite/20200831-injectdata'

## Input for script
infiles = sorted(glob.glob(join(main_dir, '*dark_dark*.fits')))[:10]
sensor_name = 'Davis_electronics'
manufacturer = 'e2v'
namps = 16
database = 'test.db'
bias_frame = None
dark_frame = None

seg_names = ['C17', 'C16', 'C15', 'C14', 'C13', 'C12', 'C11', 'C10',
             'C00', 'C01', 'C02', 'C03', 'C04', 'C05', 'C06', 'C07']

## Task Configs
ly = 200
lx = 50
num_iter = 1
nsig = 5
threshold = 1000.

In [47]:
## Get sensor information from header
all_amps = imutils.allAmps(infiles[0])
with fits.open(infiles[0]) as hdulist:
    manufacturer = hdulist[0].header['CCD_MANU']
    lsst_num = hdulist[0].header['LSST_NUM']

with db_session(database) as session:

    ## Get (add) sensor from (to) database
    try:
        sensor = Sensor.from_db(session, lsst_num=lsst_num)
    except NoResultFound:
        sensor = Sensor(sensor_name=sensor_name, lsst_num=lsst_num, manufacturer=manufacturer, 
                        namps=len(all_amps))
        sensor.segments = {i : Segment(segment_name=AMP2SEG[i], amplifier_number=i) for i in all_amps}
        sensor.add_to_db(session)
        session.commit()

    ## Get configuration and analysis settings
    
    for infile in infiles:

        ccd = MaskedCCD(infile, bias_frame=bias_frame, dark_frame=dark_frame)        
    
        for i in all_amps:
            exptime = 1
            gain = 1
            bp = BrightPixels(ccd, i, exptime, gain, ethresh=threshold)
            pixels, columns = bp.find()

            if len(columns) == 0:
                continue
            col = columns[0]
            aggressor_imarr = ccd.unbiased_and_trimmed_image(i).getImage().getArray()
            signal = np.mean(aggressor_imarr[:, col])    
            aggressor_stamp = make_stamp(aggressor_imarr, 2000, col, ly=ly, lx=lx)

            for j in all_amps:

                victim_imarr = ccd.unbiased_and_trimmed_image(j).getImage().getArray()
                victim_stamp = make_stamp(victim_imarr, 2000, col, ly=ly, lx=lx)
                res = crosstalk_fit(aggressor_stamp, victim_stamp, noise=7.0, num_iter=num_iter, 
                                    nsig=nsig)
                


In [ ]:

        aggressor_stamp = make_stamp(aggressor_imarr, 2000, col, ly=ly, lx=lx)
        if signal < threshold:
            continue

        for j in all_amps:

            victim_imarr = ccd.unbiased_and_trimmed_image(j).getImage().getArray()
            victim_stamp = make_stamp(victim_imarr, 2000, col, ly=ly, lx=lx)
            res = crosstalk_fit(aggressor_stamp, victim_stamp, noise=7.0, num_iter=num_iter, 
                                nsig=nsig)

            ## Add result to database
            result = Result(aggressor_id=sensor.segments[i].id, aggressor_signal=signal,
                            coefficient=res[0], error=res[4], method='MODEL_LSQ',
                            victim_id=sensor.segments[j].id)
            result.add_to_db(session)